# Чат рекомендаткельной системы для помощи в принятии решений 🤖:🛠

In [ ]:
!pip install transformers torch langchain -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-Coder-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# Перемещение модели на GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Получаем ввод от пользователя
data = input('Расскажите о ситуации на дороге: ')

# Токенизируем полный prompt
input_ids = tokenizer.encode(data, return_tensors='pt').to(device)  # Перемещение токенов на GPU

# Генерируем текст
output = model.generate(input_ids, max_length=100, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95)

# Декодируем сгенерированный текст
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Выводим результат
print(generated_text)

Расскажите о ситуации на дороге: г. Краснодар, улица Грибоедова образовался большой затор. Справа и слева куча стоящих машин. Как я могу улучшить ситуацию, если я могу только переключать светофор.
г. Краснодар, улица Грибоедова образовался большой затор. Справа и слева куча стоящих машин. Как я могу улучшить ситуацию, если я могу только переключать светофор. Важно сохранить безопасность на дороге.
:

. **Используйте светофор для управления потоком трафика:**
   - Переключайте светофор на желтый, чтобы предупредить водителей о том, что затор будет продолжаться. Это даст им время на подготовку к остановке или проезду через затор.
   - После того, как светофор перейдет в красный, убедитесь, что все автомобили остановились перед линией остановки.
   - Когда затор начинает разрушаться и движение становится более спокойным, начните переключать светофор на зеленый.

4. **Объясните ситуации в своем районе:**
   - Если возможно, используйте мобильное приложение или социальные сети для оповещени

# Введение проверки релевантности ответа по косинусному расстоянию 🕵️‍♂️

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Функция для выбора наиболее релевантного ответа
def choose_best_response(question, responses):
    # Создаем список, состоящий из вопроса и ответов
    documents = [question] + responses

    # Инициализируем векторизатор TF-IDF
    vectorizer = TfidfVectorizer()

    # Преобразуем документы в TF-IDF матрицу
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Вычисляем схожесть косинуса между вопросом и ответами
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Находим индекс наиболее релевантного ответа
    best_response_index = np.argmax(cosine_similarities)

    return responses[best_response_index], cosine_similarities[best_response_index]